In [7]:
# ========================================================================
# 1. IMPORTAÇÕES E CARREGAMENTO DOS DADOS
# ========================================================================
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import os
from tensorflow.keras.models import load_model
import joblib
import matplotlib.pyplot as plt

# CARREGAR AS BASES
url_daily = "https://media.githubusercontent.com/media/ruanvirginio/masters/refs/heads/main/bases_tratadas/daily_peak_transformers_dataset.csv"
df_daily = pd.read_csv(url_daily, sep=';', encoding='latin-1')

print("✅ Dados carregados com sucesso!")
print(f"Shape: {df_daily.shape}")
print(f"Transformadores únicos: {df_daily['id'].unique()}")

# ========================================================================
# 2. VERIFICAR QUE MODELOS REALMENTE EXISTEM
# ========================================================================
base_dir = r"G:\Meu Drive\Estudos\Mestrado\Github\masters\modelos"
base_dir = r"C:\users\rcvs3\OneDrive - Energisa\Documentos\Mestrado\thesis\masters\modelos"

print(f"\n🔍 Verificando arquivos no diretório: {base_dir}")

# Listar todos os arquivos .pkl no diretório
if os.path.exists(base_dir):
    arquivos = os.listdir(base_dir)
    arquivos_pkl = [f for f in arquivos if f.endswith('.pkl')]
    arquivos_h5 = [f for f in arquivos if f.endswith('.h5')]
    
    print(f"📁 Arquivos .pkl encontrados: {arquivos_pkl}")
    print(f"📁 Arquivos .h5 encontrados: {arquivos_h5}")
else:
    print(f"❌ Diretório não existe: {base_dir}")

# ========================================================================
# 3. CARREGAR APENAS OS MODELOS QUE EXISTEM
# ========================================================================
trafos = ['T21a', 'T40', 'T66', 'T4', 'T6', 'T10', 'T51', 'T57', 'T71']

modelos_carregados = {}

# Verificar cada combinação possível
for trafo in trafos:
    for modelo in ['SVR','RFR','GBR','LGBM','LSTM']:
        caminho_pkl = os.path.join(base_dir, f"{modelo}_{trafo}_daily.pkl")
        caminho_h5 = os.path.join(base_dir, f"{modelo}_{trafo}_daily.h5")
        
        if os.path.exists(caminho_pkl):
            try:
                modelos_carregados[(modelo, trafo)] = joblib.load(caminho_pkl)
                print(f"✅ Modelo carregado: {modelo}_{trafo}.pkl")
                # Verificar features
                modelo_obj = modelos_carregados[(modelo, trafo)]
                try:
                    print(f"   Features esperadas: {modelo_obj.n_features_in_}")
                except AttributeError:
                    print(f"   ✅ Modelo carregado com sucesso")
            except Exception as e:
                print(f"❌ Erro ao carregar {modelo}_{trafo}: {e}")
        elif os.path.exists(caminho_h5):
            try:
                modelos_carregados[(modelo, trafo)] = load_model(caminho_h5)
                print(f"✅ Modelo carregado: {modelo}_{trafo}.h5")
                print(f"   Input shape: {modelos_carregados[(modelo, trafo)].input_shape}")
            except Exception as e:
                print(f"❌ Erro ao carregar {modelo}_{trafo}: {e}")
        else:
            print(f"⚠️ Arquivo não encontrado: {modelo}_{trafo}")

# Listar modelos que foram carregados com sucesso
modelos_disponiveis = list(set([modelo for modelo, trafo in modelos_carregados.keys()]))
trafos_disponiveis = list(set([trafo for modelo, trafo in modelos_carregados.keys()]))

print(f"\n🎯 Modelos disponíveis para teste: {modelos_disponiveis}")
print(f"🎯 Transformadores disponíveis: {trafos_disponiveis}")

# ========================================================================
# 4. FUNÇÕES PARA TESTE
# ========================================================================
def preparar_dados_teste(df_daily, trafo, janela=365):
    """Prepara dados de teste na MESMA estrutura usada no treinamento"""
    print(f"\n📊 Preparando dados de teste para {trafo}...")
    
    # Filtrar dados do transformador específico
    df_trafo = df_daily[df_daily['id'] == trafo].copy()
    
    if len(df_trafo) == 0:
        print(f"   ❌ Nenhum dado encontrado para {trafo}!")
        return None, None, None
    
    df_trafo = df_trafo[['datahora', 'S']].set_index('datahora').sort_index()
    
    print(f"   Dados disponíveis para {trafo}: {len(df_trafo)} registros")
    
    # Verificar se temos dados suficientes
    if len(df_trafo) < janela + 10:
        print(f"   ❌ Dados insuficientes para {trafo}. Necessário: {janela + 10}, Disponível: {len(df_trafo)}")
        return None, None, None
    
    # Separar os últimos 30% para teste
    split_idx = int(len(df_trafo) * 0.7)
    df_test = df_trafo.iloc[split_idx:]
    
    print(f"   Dados de teste: {len(df_test)} registros")
    print(f"   Período de teste: {df_test.index[0]} até {df_test.index[-1]}")
    
    # Criar janelas deslizantes
    X_test, y_test = [], []
    for i in range(janela, len(df_test)):
        X_test.append(df_test.iloc[i-janela:i].values)
        y_test.append(df_test.iloc[i, 0])
    
    if len(X_test) == 0:
        print(f"   ❌ Não foi possível criar janelas para {trafo}")
        return None, None, None
    
    X_test = np.array(X_test)
    y_test = np.array(y_test)
    
    print(f"   Shape final X_test: {X_test.shape}")
    print(f"   Número de amostras de teste: {len(y_test)}")
    
    return X_test, y_test, df_test.index[janela:]

def testar_modelo(modelo_nome, trafo_nome, X_test, y_test, datas_test):
    """Testa um modelo específico com dados preparados"""
    print(f"\n🧪 Testando {modelo_nome} - {trafo_nome}...")
    
    modelo = modelos_carregados.get((modelo_nome, trafo_nome))
    if modelo is None:
        print(f"   ❌ Modelo não encontrado!")
        return None
    
    try:
        # Fazer previsões - todos os modelos atuais precisam dos dados achatados
        X_test_flat = X_test.reshape(X_test.shape[0], -1)
        y_pred = modelo.predict(X_test_flat)
        
        # Calcular métricas
        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        rmse = np.sqrt(mse)
        
        print(f"   ✅ Previsões realizadas!")
        print(f"   📊 Métricas de TESTE:")
        print(f"      - RMSE: {rmse:.4f}")
        print(f"      - MAE:  {mae:.4f}")
        print(f"      - R²:   {r2:.4f}")
        
        # Mostrar algumas previsões vs reais
        print(f"   🔍 Amostra de previsões (3 primeiras):")
        for i in range(min(3, len(y_test))):
            print(f"      Real: {y_test[i]:.4f} | Previsto: {y_pred[i]:.4f} | Dif: {abs(y_test[i] - y_pred[i]):.4f}")
        
        return {
            'y_real': y_test,
            'y_pred': y_pred,
            'datas': datas_test,
            'metrics': {'RMSE': rmse, 'MAE': mae, 'R2': r2}
        }
        
    except Exception as e:
        print(f"   ❌ Erro ao testar modelo: {e}")
        return None

# ========================================================================
# 5. EXECUTAR TESTES APENAS COM O QUE TEMOS
# ========================================================================
print("\n" + "="*60)
print("🚀 INICIANDO TESTES COM MODELOS DISPONÍVEIS")
print("="*60)

# Armazenar todos os resultados
resultados_comparacao = []

for trafo in trafos_disponiveis:
    print(f"\n{'='*50}")
    print(f"🎯 TESTANDO TRANSFORMADOR: {trafo}")
    print(f"{'='*50}")
    
    # Preparar dados de teste para este trafo
    X_test, y_test, datas_test = preparar_dados_teste(df_daily, trafo, janela=30)
    
    if X_test is None:
        continue
    
    # Testar cada modelo disponível para este trafo
    for modelo_nome in modelos_disponiveis:
        if (modelo_nome, trafo) in modelos_carregados:
            resultado = testar_modelo(modelo_nome, trafo, X_test, y_test, datas_test)
            
            if resultado is not None:
                # Adicionar à comparação
                resultados_comparacao.append({
                    'Modelo': modelo_nome,
                    'Trafo': trafo,
                    'RMSE': resultado['metrics']['RMSE'],
                    'MAE': resultado['metrics']['MAE'],
                    'R2': resultado['metrics']['R2']
                })

# ========================================================================
# 6. MOSTRAR RESULTADOS FINAIS
# ========================================================================
if resultados_comparacao:
    print("\n" + "="*60)
    print("📊 RESULTADOS FINAIS")
    print("="*60)
    
    df_resultados = pd.DataFrame(resultados_comparacao)
    print("\n" + df_resultados.round(4).to_string(index=False))
    
    # Salvar resultados
    os.makedirs('resultados', exist_ok=True)
    df_resultados.to_csv('resultados/resultados_testes.csv', index=False)
    print(f"\n💾 Resultados salvos em: resultados/resultados_testes.csv")
else:
    print("\n❌ Nenhum modelo foi testado com sucesso.")

print("\n🎉 PROCESSO CONCLUÍDO!")

✅ Dados carregados com sucesso!
Shape: (218337, 3)
Transformadores únicos: ['T1' nan 'T10' 'T11a' 'T11b' 'T12' 'T13a' 'T13b' 'T14' 'T15' 'T16a'
 'T16b' 'T17' 'T18' 'T19a' 'T19b' 'T2' 'T20a' 'T20b' 'T21a' 'T21b' 'T22a'
 'T22b' 'T23' 'T24' 'T25' 'T26' 'T27' 'T28a' 'T28b' 'T29' 'T3' 'T30a'
 'T30b' 'T31' 'T32' 'T33' 'T34' 'T35' 'T36a' 'T36b' 'T37' 'T38' 'T39' 'T4'
 'T40' 'T41' 'T42a' 'T42b' 'T43' 'T44' 'T45' 'T46a' 'T46b' 'T47' 'T48'
 'T49' 'T5' 'T50' 'T51' 'T52a' 'T52b' 'T52c' 'T53' 'T54' 'T55' 'T56' 'T57'
 'T58' 'T59' 'T6' 'T60a' 'T60b' 'T61' 'T62' 'T63' 'T64' 'T65' 'T66' 'T67'
 'T68a' 'T68b' 'T69a' 'T69b' 'T70a' 'T70b' 'T71' 'T72' 'T7a' 'T7b' 'T8'
 'T9']

🔍 Verificando arquivos no diretório: C:\users\rcvs3\OneDrive - Energisa\Documentos\Mestrado\thesis\masters\modelos
📁 Arquivos .pkl encontrados: ['GBR_T10_daily.pkl', 'GBR_T1_daily.pkl', 'GBR_T21a_daily.pkl', 'GBR_T2_daily.pkl', 'GBR_T40_daily.pkl', 'GBR_T4_daily.pkl', 'GBR_T51_daily.pkl', 'GBR_T57_daily.pkl', 'GBR_T66_daily.pkl', 'GBR_